In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
os.chdir("/kaggle/input/col341-a3")

In [3]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import cv2
from time import time
# Import TF and TF Hub libraries.
import tensorflow as tf
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# import torchvision
# from torch.utils.data import Dataset, DataLoader
np.random.seed(46)
# torch.manual_seed(46)
# from torchvision import transforms, utils,models
#from efficientnet_pytorch import EfficientNet
# from pytorch_pretrained_vit import ViT
import keras
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
tf.random.set_seed(46)

In [4]:
train_data=pd.read_csv('training.csv')
train_data.head()

,name,category
0,./training/450030N425231.jpg,Virabhadrasana
1,./training/2634j01014741.jpg,Vrikshasana
2,./training/440023N423072.jpg,Utkatasana
3,./training/2845016N420103.jpg,Padahastasana
4,./training/100024N423211.jpg,Katichakrasana


In [5]:
train_data['category']=train_data["category"].astype('category').cat.codes
train_data.head()

,name,category
0,./training/450030N425231.jpg,17
1,./training/2634j01014741.jpg,18
2,./training/440023N423072.jpg,16
3,./training/2845016N420103.jpg,7
4,./training/100024N423211.jpg,3


In [6]:
train_data=train_data.sample(frac=1.,random_state=46)
train_data,test_data=train_data.iloc[:29000],train_data.iloc[29000:]
train_data.shape,test_data.shape

((29000, 2), (11, 2))

In [7]:
import tensorflow, gc
from keras.preprocessing.image import ImageDataGenerator

In [8]:
class DataGenerator(tensorflow.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, data,train=True,val=False, batch_size=32, n_channels=3,
                 n_classes=19, shuffle=True):
        'Initialization'
        #self.dim = dim
        self.train=train
        self.val=val
        self.batch_size = batch_size
        if(self.train):
            self.labels = data.iloc[:,1].to_numpy()
            self.list_IDs = data.iloc[:,0].to_numpy()
        else:
            self.labels = data.to_numpy().squeeze()
            self.list_IDs = data.to_numpy().squeeze()
        self.total=self.labels.shape[0] 
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()
        self.augmentor = ImageDataGenerator(
            rotation_range=15,
            width_shift_range=0.15,
            height_shift_range=0.15,
            shear_range=0.15,
            zoom_range=0.15,
            fill_mode='nearest',
            horizontal_flip=True,
        )
        #Image.fromarray(numpy_image.astype('uint8'), 'RGB')
    def __len__(self):
        'Denotes the number of batches per epoch'
        if (self.train==False):
            return int(np.floor(len(self.list_IDs) / self.batch_size))+1
        else:
            return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
    
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
       
        # Find list of IDs
        image_names = [self.list_IDs[k] for k in indexes]
        if(self.train):
            y=np.array([self.labels[i] for i in indexes],dtype=int)
        # Generate data
        X = self.__data_generation(image_names)
        if(self.train):
            return X, tensorflow.keras.utils.to_categorical(y, num_classes=self.n_classes)
        else:
            return X,np.array(image_names)

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        gc.collect()
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        os.chdir("/kaggle/input/col341-a3")
        
        X = np.empty((self.batch_size, 256,256, self.n_channels))
        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            X[i,] = cv2.imread(ID)[-256:,128:128+256,:]
        if(self.train==True and self.val==False):
            X = self.augmentor.flow(X, batch_size=self.batch_size, shuffle=False)[0]
#         X=np.array(transforms.Compose([transforms.ToPILImage(),transforms.RandomPerspective()])(X))
        gc.collect()
        os.chdir("/kaggle/working")
        return X

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow import keras as keras


# Generators
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow import keras as keras



def train():
    training_generator = DataGenerator(train_data,True,False,24,3,19,True)
    validation_generator = DataGenerator(test_data,True,True,24,3,19,True)


    base_model = tf.keras.applications.EfficientNetB5(
        include_top=False,
        weights="imagenet",
        classes=19,
    )

    inputs = keras.Input(shape=(256, 256, 3))

    x = base_model(inputs, training=True)
    # x = keras.layers.Flatten()(x)
    x = keras.layers.GlobalMaxPool2D()(x)
    x = keras.layers.Dense(1024,activation='relu')(x)
    #x = keras.layers.Dense(128,activation='relu')(x)
    outputs = keras.layers.Dense(19,activation='softmax')(x)
    model = keras.Model(inputs, outputs)
    gc.collect()


    def scheduler(epoch, lr):
        return lr*((.9)**(epoch-1))
    modelcheckpoint=tf.keras.callbacks.ModelCheckpoint(
        "model.h5",
        monitor="val_acc",
        verbose=0,
        save_best_only=True,
        save_weights_only=True,
        mode="auto",
        save_freq="epoch",
        options=None,
    )
    #tf.config.get_visible_devices()
    lrs = keras.callbacks.LearningRateScheduler(scheduler)
    model.compile(optimizer=optimizers.Adam(learning_rate=6e-4),
              loss='categorical_crossentropy',metrics=["acc"])
    # Train model on dataset
    model.fit_generator(epochs=5,generator=training_generator,
                        validation_data=validation_generator,
                        use_multiprocessing=True,callbacks=[lrs],
                        workers=6)

    os.chdir('/kaggle/working')
    model.save('effb5')

    import shutil
    shutil.make_archive('effb5', 'zip', 'effb5')

In [10]:
import multiprocessing


p = multiprocessing.Process(target=train)
p.start()
p.join()

2021-11-30 08:23:36.766130: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-30 08:23:36.767748: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-30 08:23:36.768843: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-30 08:23:36.770125: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

115277824/115263384 [==============================] - 1s 0us/step


/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5


2021-11-30 08:23:49.621498: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-30 08:24:14.852967: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


1208/1208 [==============================] - 980s 785ms/step - loss: 0.8668 - acc: 0.7590
Epoch 2/5
1208/1208 [==============================] - 957s 787ms/step - loss: 0.4486 - acc: 0.8611
Epoch 3/5
1208/1208 [==============================] - 948s 780ms/step - loss: 0.3687 - acc: 0.8795
Epoch 4/5
1208/1208 [==============================] - 960s 791ms/step - loss: 0.3086 - acc: 0.8964
Epoch 5/5
1208/1208 [==============================] - 964s 794ms/step - loss: 0.2556 - acc: 0.9108


2021-11-30 09:46:16.981660: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [11]:
import os
os.chdir("/kaggle/input/col341-a3")

In [12]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import cv2
from time import time
# Import TF and TF Hub libraries.
import tensorflow as tf
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# import torchvision
# from torch.utils.data import Dataset, DataLoader
np.random.seed(46)
# torch.manual_seed(46)
# from torchvision import transforms, utils,models
#from efficientnet_pytorch import EfficientNet
# from pytorch_pretrained_vit import ViT
import keras
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
tf.random.set_seed(46)
import random
random.seed(6)

In [13]:
train_data=pd.read_csv('training.csv')
train_data.head()

,name,category
0,./training/450030N425231.jpg,Virabhadrasana
1,./training/2634j01014741.jpg,Vrikshasana
2,./training/440023N423072.jpg,Utkatasana
3,./training/2845016N420103.jpg,Padahastasana
4,./training/100024N423211.jpg,Katichakrasana


In [14]:
train_data['category']=train_data["category"].astype('category').cat.codes
train_data.head()

,name,category
0,./training/450030N425231.jpg,17
1,./training/2634j01014741.jpg,18
2,./training/440023N423072.jpg,16
3,./training/2845016N420103.jpg,7
4,./training/100024N423211.jpg,3


In [15]:
train_data=train_data.sample(frac=1.,random_state=46)
train_data,test_data=train_data.iloc[:29000],train_data.iloc[29000:]
train_data.shape,test_data.shape
# train_data=train_data.to_numpy()
# train_data.shape
# np.random.shuffle(train_data)
# train_data,test_data=train_data[:26000],train_data[26000:]
# train_data.shape,test_data.shape

((29000, 2), (11, 2))

In [16]:
import tensorflow, gc
from keras.preprocessing.image import ImageDataGenerator

In [17]:
class DataGenerator(tensorflow.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, data,train=True,val=False, batch_size=32, n_channels=3,
                 n_classes=19, shuffle=True):
        'Initialization'
        #self.dim = dim
        self.train=train
        self.val=val
        self.batch_size = batch_size
        if(self.train):
            self.labels = data.iloc[:,1].to_numpy()
            self.list_IDs = data.iloc[:,0].to_numpy()
        else:
            self.labels = data.to_numpy().squeeze()
            self.list_IDs = data.to_numpy().squeeze()
        self.total=self.labels.shape[0] 
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()
        self.augmentor = ImageDataGenerator(
            rotation_range=15,
            width_shift_range=0.15,
            height_shift_range=0.15,
            shear_range=0.15,
            zoom_range=0.15,
            fill_mode='nearest',
            horizontal_flip=True,
        )
        #Image.fromarray(numpy_image.astype('uint8'), 'RGB')
    def __len__(self):
        'Denotes the number of batches per epoch'
        if (self.train==False):
            return int(np.floor(len(self.list_IDs) / self.batch_size))+1
        else:
            return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
    
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
       
        # Find list of IDs
        image_names = [self.list_IDs[k] for k in indexes]
        if(self.train):
            y=np.array([self.labels[i] for i in indexes],dtype=int)
        # Generate data
        X = self.__data_generation(image_names)
        if(self.train):
            return X, tensorflow.keras.utils.to_categorical(y, num_classes=self.n_classes)
        else:
            return X,np.array(image_names)

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        gc.collect()
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        os.chdir("/kaggle/input/col341-a3")
        
        X = np.empty((self.batch_size, 256,256, self.n_channels))
        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            X[i,] = cv2.imread(ID)[-256:,128:128+256,:]
        if(self.train==True and self.val==False):
            X = self.augmentor.flow(X, batch_size=self.batch_size, shuffle=False)[0]
        gc.collect()
        os.chdir("/kaggle/working")
        return X

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow import keras as keras


# Generators


def train():
    training_generator = DataGenerator(train_data,True,False,40,3,19,True)
    validation_generator = DataGenerator(test_data,True,True,40,3,19,True)

    base_model = tf.keras.applications.Xception(
        include_top=False,
        weights="imagenet",
        classes=19,
    )

    inputs = keras.Input(shape=(256, 256, 3))

    x = base_model(inputs, training=True)
    # x = keras.layers.Flatten()(x)
    x = keras.layers.GlobalMaxPool2D()(x)
    x = keras.layers.Dense(1024,activation='relu')(x)
    #x = keras.layers.Dense(128,activation='relu')(x)
    outputs = keras.layers.Dense(19,activation='softmax')(x)
    model = keras.Model(inputs, outputs)
    gc.collect()

    def scheduler(epoch, lr):
        return lr*((.95)**(epoch-1))
    modelcheckpoint=tf.keras.callbacks.ModelCheckpoint(
        "model.h5",
        monitor="val_acc",
        verbose=0,
        save_best_only=True,
        save_weights_only=True,
        mode="auto",
        save_freq="epoch",
        options=None,
    )
    #tf.config.get_visible_devices()
    lrs = keras.callbacks.LearningRateScheduler(scheduler)
    model.compile(optimizer=optimizers.Adam(learning_rate=5.5e-4),
              loss='categorical_crossentropy',metrics=["acc"])
    # Train model on dataset
    model.fit_generator(epochs=5,generator=training_generator,
                        validation_data=validation_generator,
                        use_multiprocessing=True,callbacks=[lrs],
                        workers=6)

    os.chdir('/kaggle/working')
    model.save('xception')

    import shutil
    shutil.make_archive('xception', 'zip', 'xception')

In [19]:
import multiprocessing


p = multiprocessing.Process(target=train)
p.start()
p.join()

2021-11-30 09:49:31.465771: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-30 09:49:31.467459: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-30 09:49:31.468347: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-30 09:49:31.469591: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

83697664/83683744 [==============================] - 0s 0us/step


/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5


2021-11-30 09:49:36.788207: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-30 09:49:47.917805: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


725/725 [==============================] - 642s 864ms/step - loss: 1.2593 - acc: 0.6005
Epoch 2/5
725/725 [==============================] - 595s 804ms/step - loss: 0.4842 - acc: 0.8483
Epoch 3/5
725/725 [==============================] - 602s 822ms/step - loss: 0.3917 - acc: 0.8735
Epoch 4/5
725/725 [==============================] - 616s 834ms/step - loss: 0.3347 - acc: 0.8889
Epoch 5/5
725/725 [==============================] - 621s 842ms/step - loss: 0.2835 - acc: 0.9019


2021-11-30 10:42:34.464671: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [20]:
import os
os.chdir("/kaggle/input/col341-a3")

In [21]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import cv2
from time import time
# Import TF and TF Hub libraries.
import tensorflow as tf
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# import torchvision
# from torch.utils.data import Dataset, DataLoader
np.random.seed(46)
# torch.manual_seed(46)
# from torchvision import transforms, utils,models
#from efficientnet_pytorch import EfficientNet
# from pytorch_pretrained_vit import ViT
import keras
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
tf.random.set_seed(46)
# import random
# random.seed(14)

In [22]:
train_data=pd.read_csv('training.csv')
train_data.head()
train_data['category']=train_data["category"].astype('category').cat.codes
train_data.head()
train_data=train_data.sample(frac=1.,random_state=46)
train_data,test_data=train_data.iloc[:29000],train_data.iloc[29000:]
train_data.shape,test_data.shape

((29000, 2), (11, 2))

In [23]:
import tensorflow, gc
from keras.preprocessing.image import ImageDataGenerator
class DataGenerator(tensorflow.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, data,train=True,val=False, batch_size=32, n_channels=3,
                 n_classes=19, shuffle=True):
        'Initialization'
        #self.dim = dim
        self.train=train
        self.val=val
        self.batch_size = batch_size
        if(self.train):
            self.labels = data.iloc[:,1].to_numpy()
            self.list_IDs = data.iloc[:,0].to_numpy()
        else:
            self.labels = data.to_numpy().squeeze()
            self.list_IDs = data.to_numpy().squeeze()
        self.total=self.labels.shape[0] 
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()
        self.augmentor = ImageDataGenerator(
            rotation_range=15,
            width_shift_range=0.15,
            height_shift_range=0.15,
            shear_range=0.15,
            zoom_range=0.15,
            fill_mode='nearest',
            horizontal_flip=True,
        )
        #Image.fromarray(numpy_image.astype('uint8'), 'RGB')
    def __len__(self):
        'Denotes the number of batches per epoch'
        if (self.train==False):
            return int(np.floor(len(self.list_IDs) / self.batch_size))+1
        else:
            return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
    
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
       
        # Find list of IDs
        image_names = [self.list_IDs[k] for k in indexes]
        if(self.train):
            y=np.array([self.labels[i] for i in indexes],dtype=int)
        # Generate data
        X = self.__data_generation(image_names)
        if(self.train):
            return X, tensorflow.keras.utils.to_categorical(y, num_classes=self.n_classes)
        else:
            return X,np.array(image_names)

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        gc.collect()
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        os.chdir("/kaggle/input/col341-a3")
        
        X = np.empty((self.batch_size, 256,256, self.n_channels))
        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            X[i,] = cv2.imread(ID)[-256:,128:128+256,:]
        if(self.train==True and self.val==False):
            X = self.augmentor.flow(X, batch_size=self.batch_size, shuffle=False)[0]
        gc.collect()
        os.chdir("/kaggle/working")
        return X

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow import keras as keras


# Generators
def train():
    training_generator = DataGenerator(train_data,True,False,40,3,19,True)
    validation_generator = DataGenerator(test_data,True,True,40,3,19,True)


    base_model = tf.keras.applications.ResNet50V2(
        include_top=False,
        weights="imagenet",
        classes=19,
    )

    inputs = keras.Input(shape=(256, 256, 3))

    x = base_model(inputs, training=True)
    # x = keras.layers.Flatten()(x)
    x = keras.layers.GlobalMaxPool2D()(x)
    x = keras.layers.Dense(1024,activation='relu')(x)
    #x = keras.layers.Dense(128,activation='relu')(x)
    outputs = keras.layers.Dense(19,activation='softmax')(x)
    model = keras.Model(inputs, outputs)
    gc.collect()

    def scheduler(epoch, lr):
        return lr*((.95)**(epoch-1))
    modelcheckpoint=tf.keras.callbacks.ModelCheckpoint(
        "model.h5",
        monitor="val_acc",
        verbose=0,
        save_best_only=True,
        save_weights_only=True,
        mode="auto",
        save_freq="epoch",
        options=None,
    )
    #tf.config.get_visible_devices()
    lrs = keras.callbacks.LearningRateScheduler(scheduler)
    model.compile(optimizer=optimizers.Adam(learning_rate=5e-4),
              loss="categorical_crossentropy",metrics=["acc"])

    # Train model on dataset
    model.fit_generator(epochs=5,generator=training_generator,
                        validation_data=validation_generator,
                        use_multiprocessing=True,callbacks=[lrs],
                        workers=6)

    os.chdir('/kaggle/working')
    model.save('resnet50v2')

    import shutil
    shutil.make_archive('resnet50v2', 'zip', 'resnet50v2')
    

In [25]:
import multiprocessing

p = multiprocessing.Process(target=train)
p.start()
p.join()

2021-11-30 10:45:10.819250: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-30 10:45:10.821131: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-30 10:45:10.822218: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-30 10:45:10.823663: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

94683136/94668760 [==============================] - 1s 0us/step


/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5


2021-11-30 10:45:16.822982: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-30 10:45:28.068176: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


725/725 [==============================] - 613s 824ms/step - loss: 1.1550 - acc: 0.6978
Epoch 2/5
725/725 [==============================] - 577s 778ms/step - loss: 0.5633 - acc: 0.8241
Epoch 3/5
725/725 [==============================] - 573s 782ms/step - loss: 0.4781 - acc: 0.8500
Epoch 4/5
725/725 [==============================] - 573s 784ms/step - loss: 0.4219 - acc: 0.8635
Epoch 5/5
725/725 [==============================] - 576s 777ms/step - loss: 0.3775 - acc: 0.8739


2021-11-30 11:35:45.780705: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [26]:
import os
os.chdir("/kaggle/input/col341-a3")
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import cv2
from time import time
# Import TF and TF Hub libraries.
import tensorflow as tf
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# import torchvision
# from torch.utils.data import Dataset, DataLoader
np.random.seed(46)
# torch.manual_seed(46)
# from torchvision import transforms, utils,models
#from efficientnet_pytorch import EfficientNet
# from pytorch_pretrained_vit import ViT
import keras
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
tf.random.set_seed(46)
import random
random.seed(6)

In [27]:
train_data=pd.read_csv('training.csv')
train_data.head()
train_data['category']=train_data["category"].astype('category').cat.codes
train_data.head()
train_data=train_data.sample(frac=1.,random_state=46)
train_data,test_data=train_data.iloc[:29000],train_data.iloc[29000:]
train_data.shape,test_data.shape

((29000, 2), (11, 2))

In [28]:
import tensorflow, gc
from keras.preprocessing.image import ImageDataGenerator
class DataGenerator(tensorflow.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, data,train=True,val=False, batch_size=32, n_channels=3,
                 n_classes=19, shuffle=True):
        'Initialization'
        #self.dim = dim
        self.train=train
        self.val=val
        self.batch_size = batch_size
        if(self.train):
            self.labels = data.iloc[:,1].to_numpy()
            self.list_IDs = data.iloc[:,0].to_numpy()
        else:
            self.labels = data.to_numpy().squeeze()
            self.list_IDs = data.to_numpy().squeeze()
        self.total=self.labels.shape[0] 
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()
        self.augmentor = ImageDataGenerator(
            rotation_range=15,
            width_shift_range=0.15,
            height_shift_range=0.15,
            shear_range=0.15,
            zoom_range=0.15,
            fill_mode='nearest',
            horizontal_flip=True,
        )
        #Image.fromarray(numpy_image.astype('uint8'), 'RGB')
    def __len__(self):
        'Denotes the number of batches per epoch'
        if (self.train==False):
            return int(np.floor(len(self.list_IDs) / self.batch_size))+1
        else:
            return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
    
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
       
        # Find list of IDs
        image_names = [self.list_IDs[k] for k in indexes]
        if(self.train):
            y=np.array([self.labels[i] for i in indexes],dtype=int)
        # Generate data
        X = self.__data_generation(image_names)
        if(self.train):
            return X, tensorflow.keras.utils.to_categorical(y, num_classes=self.n_classes)
        else:
            return X,np.array(image_names)

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        gc.collect()
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        os.chdir("/kaggle/input/col341-a3")
        
        X = np.empty((self.batch_size, 256,256, self.n_channels))
        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            X[i,] = cv2.imread(ID)[-256:,128:128+256,:]
        if(self.train==True and self.val==False):
            X = self.augmentor.flow(X, batch_size=self.batch_size, shuffle=False)[0]
        gc.collect()
        os.chdir("/kaggle/working")
        return X

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow import keras as keras


# Generators
def train():
    training_generator = DataGenerator(train_data,True,False,40,3,19,True)
    validation_generator = DataGenerator(test_data,True,True,40,3,19,True)


    base_model = tf.keras.applications.EfficientNetB4(
        include_top=False,
        weights="imagenet",
        classes=19,
    )

    inputs = keras.Input(shape=(256, 256, 3))

    x = base_model(inputs, training=True)
    # x = keras.layers.Flatten()(x)
    x = keras.layers.GlobalMaxPool2D()(x)
    x = keras.layers.Dense(1024,activation='relu')(x)
    #x = keras.layers.Dense(128,activation='relu')(x)
    outputs = keras.layers.Dense(19,activation='softmax')(x)
    model = keras.Model(inputs, outputs)
    gc.collect()

    def scheduler(epoch, lr):
        return lr*((.95)**(epoch-1))
    modelcheckpoint=tf.keras.callbacks.ModelCheckpoint(
        "model.h5",
        monitor="val_acc",
        verbose=0,
        save_best_only=True,
        save_weights_only=True,
        mode="auto",
        save_freq="epoch",
        options=None,
    )
    #tf.config.get_visible_devices()
    lrs = keras.callbacks.LearningRateScheduler(scheduler)
    model.compile(optimizer=optimizers.Adam(learning_rate=5e-4),
              loss='categorical_crossentropy',metrics=["acc"])

    # Train model on dataset
    model.fit_generator(epochs=5,generator=training_generator,
                        validation_data=validation_generator,
                        use_multiprocessing=True,callbacks=[lrs],
                        workers=6)

    os.chdir('/kaggle/working')
    model.save('effb4')

    import shutil
    shutil.make_archive('effb4', 'zip', 'effb4')

In [30]:
import multiprocessing

p = multiprocessing.Process(target=train)
p.start()
p.join()

2021-11-30 11:38:43.365796: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-30 11:38:43.367513: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-30 11:38:43.368367: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-30 11:38:43.369515: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

71696384/71686520 [==============================] - 0s 0us/step


/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5


2021-11-30 11:38:52.120695: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-30 11:39:15.431616: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


725/725 [==============================] - 756s 1s/step - loss: 0.8749 - acc: 0.7694
Epoch 2/5
725/725 [==============================] - 705s 963ms/step - loss: 0.3797 - acc: 0.8761
Epoch 3/5
725/725 [==============================] - 717s 968ms/step - loss: 0.3232 - acc: 0.8915
Epoch 4/5
725/725 [==============================] - 726s 984ms/step - loss: 0.2756 - acc: 0.9043
Epoch 5/5
725/725 [==============================] - 716s 978ms/step - loss: 0.2310 - acc: 0.9181


2021-11-30 12:41:10.032906: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [31]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import cv2
from time import time
# Import TF and TF Hub libraries.
import tensorflow as tf
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# import torchvision
# from torch.utils.data import Dataset, DataLoader
np.random.seed(46)
# torch.manual_seed(46)
# from torchvision import transforms, utils,models
#from efficientnet_pytorch import EfficientNet
# from pytorch_pretrained_vit import ViT
import keras
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
tf.random.set_seed(46)


In [32]:
class DataGenerator(tensorflow.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, data,train=True,val=False, batch_size=32, n_channels=3,
                 n_classes=19, shuffle=True):
        'Initialization'
        #self.dim = dim
        self.train=train
        self.val=val
        self.batch_size = batch_size
        if(self.train):
            self.labels = data.iloc[:,1].to_numpy()
            self.list_IDs = data.iloc[:,0].to_numpy()
        else:
            self.labels = data.to_numpy().squeeze()
            self.list_IDs = data.to_numpy().squeeze()
        self.total=self.labels.shape[0] 
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()
        self.augmentor = ImageDataGenerator(
            rotation_range=15,
            width_shift_range=0.15,
            height_shift_range=0.15,
            shear_range=0.15,
            zoom_range=0.15,
            fill_mode='nearest',
            horizontal_flip=True,
        )
        #Image.fromarray(numpy_image.astype('uint8'), 'RGB')
    def __len__(self):
        'Denotes the number of batches per epoch'
        if (self.train==False):
            return int(np.floor(len(self.list_IDs) / self.batch_size))+1
        else:
            return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
    
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
       
        # Find list of IDs
        image_names = [self.list_IDs[k] for k in indexes]
        if(self.train):
            y=np.array([self.labels[i] for i in indexes],dtype=int)
        # Generate data
        X = self.__data_generation(image_names)
        if(self.train):
            return X, tensorflow.keras.utils.to_categorical(y, num_classes=self.n_classes)
        else:
            return X,np.array(image_names)

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        gc.collect()
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        os.chdir("/kaggle/input/col341-a3")
        
        X = np.empty((self.batch_size, 256,256, self.n_channels))
        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            X[i,] = cv2.imread(ID)[-256:,128:128+256,:]
        if(self.train==True and self.val==False):
            X = self.augmentor.flow(X, batch_size=self.batch_size, shuffle=False)[0]
        gc.collect()
        os.chdir("/kaggle/working")
        return X

In [33]:
os.chdir("/kaggle/input/col341-a3")
private_test_data=pd.read_csv('test.csv')

In [34]:
private_test_data.shape

(8800, 1)

In [35]:
d={0:'Ardhachakrasana', 1:'Garudasana', 2:'Gorakshasana', 3:'Katichakrasana',
       4:'Natarajasana', 5:'Natavarasana', 6:'Naukasana', 7:'Padahastasana',
       8:'ParivrittaTrikonasana', 9:'Pranamasana', 10:'Santolanasana',11: 'Still',
       12:'Tadasana', 13:'Trikonasana', 14:'TriyakTadasana', 15:'Tuladandasana',
       16:'Utkatasana', 17:'Virabhadrasana',18: 'Vrikshasana'}

In [36]:
private_test_data.head()

,name
0,./testing/100032N39150.jpg
1,./testing/1156016N33502.jpg
2,./testing/1283038N310628.jpg
3,./testing/2232023N36263.jpg
4,./testing/2404020N34858.jpg


In [37]:
private_test=DataGenerator(private_test_data,False,False,39,shuffle=False)

In [38]:
os.chdir('/kaggle/working')
model1=tf.keras.models.load_model('effb4')
model2=tf.keras.models.load_model('resnet50v2')
model3=tf.keras.models.load_model('xception')
model4=tf.keras.models.load_model('effb5')
os.chdir("/kaggle/input/col341-a3")

2021-11-30 12:47:10.753356: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-30 12:47:10.754365: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-30 12:47:10.755050: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-30 12:47:10.755994: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [39]:
preds1=model1.predict_generator(private_test)
preds2=model2.predict_generator(private_test)
preds3=model3.predict_generator(private_test)
preds4=model4.predict_generator(private_test)
# preds5=model5.predict_generator(private_test)

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '
2021-11-30 12:49:15.625577: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-30 12:49:20.839023: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


In [40]:
preds1=preds1[:private_test_data.shape[0]]
preds2=preds2[:private_test_data.shape[0]]
preds3=preds3[:private_test_data.shape[0]]
preds4=preds4[:private_test_data.shape[0]]
# preds5=preds5[:private_test_data.shape[0]]

In [41]:
os.chdir('/kaggle/working')
np.save('ens_f2-effb4.npy',preds1)
np.save('ens_f2-resnet50v2.npy',preds2)
np.save('ens_f2-xception.npy',preds3)
np.save('ens_f2-effb5.npy',preds4)
os.chdir("/kaggle/input/col341-a3")

In [42]:
preds=preds1*6.3+preds2*6.3+preds3*6.3+preds4*6
preds.shape

(8800, 19)

In [43]:
preds=np.argmax(preds,axis=1)
preds.shape

(8800,)

In [44]:
submission=[]
names=private_test_data['name'].to_numpy().squeeze()
for i in range(preds.shape[0]):
    submission.append([names[i],d[preds[i]]])

In [45]:
submission=np.array(submission)
submission.shape

(8800, 2)

In [46]:
submission = pd.DataFrame(submission, columns = ['name','category']).iloc[:-1]

In [47]:
os.chdir('/kaggle/working')
submission.to_csv('submission.csv',index=False)
submission.shape

(8799, 2)

In [48]:
submission.head()

,name,category
0,./testing/100032N39150.jpg,Still
1,./testing/1156016N33502.jpg,Natarajasana
2,./testing/1283038N310628.jpg,Katichakrasana
3,./testing/2232023N36263.jpg,Padahastasana
4,./testing/2404020N34858.jpg,Padahastasana
